Imports

In [1]:
#imports
from tqdm import tqdm

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, pipeline, Trainer, TrainingArguments

from bert_training import make_training_pairs, gold_lookup

from utils import load_data, prepare_all_samples, get_entity_date_pairs, calculate_metrics
from naive_extractor import naive_extraction
from bert_extractor import get_context_window, mark_entity, mark_date, preprocess_input, bert_extraction
from llm_extractor import load_prompt_template, make_binary_prompt, llm_extraction, parse_llm_answer

Data Loading

In [2]:
# Load data
df = load_data("data/synthetic.csv")
print(f"Loaded {len(df)} records")
#df

Loaded 101 records


In [3]:
# Prepare all samples
samples = prepare_all_samples(df)
print(f"Prepared {len(samples)} samples")
#samples[0]

Prepared 101 samples


Naive Extractor

In [ ]:
#Process samples
predictions = []

for sample in samples:
    relationships = naive_extraction(sample['entities_list'], sample['dates'])
    predictions.extend(relationships)

In [ ]:
# Look at predictions
#predictions

In [ ]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

BERT Base

In [15]:
# Load local model and tokenizer
#model_path = "./bert_model_training/base_model"
model_path = "google/bert_uncased_L-2_H-128_A-2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#Example note
note_text = (
    "Patient diagnosed with asthma on 2024-08-02. "
    "Diabetes was ruled out on 2024-08-02. "
    "Family history of hypertension, last reviewed in 2022. "
    "Patient may have pneumonia, last seen on 2024-08-02."
)

# Example entity and date spans (positions are for illustration)
entity = {'start': 23, 'end': 29, 'label': 'asthma'}  # "asthma"
date = {'start': 33, 'end': 43, 'parsed': '2024-08-02'}  # "2024-08-02"

note_text, entity, date

('Patient diagnosed with asthma on 2024-08-02. Diabetes was ruled out on 2024-08-02. Family history of hypertension, last reviewed in 2022. Patient may have pneumonia, last seen on 2024-08-02.',
 {'start': 23, 'end': 29, 'label': 'asthma'},
 {'start': 33, 'end': 43, 'parsed': '2024-08-02'})

In [8]:
#Test context window
context = get_context_window(note_text, entity['start'], date['start'], window_size=50)
print("Context window:\n", context)

Context window:
 Patient diagnosed with asthma on 2024-08-02. Diabetes was ruled out on 2024-08-02. 


In [10]:
#Test entity marking
entity_text = note_text[entity['start']:entity['end']]
offset = context.find(entity_text)
entity_rel = {'start': offset, 'end': offset + len(entity_text)}

marked_entity = mark_entity(context, entity_rel)
print("Entity marked:\n", marked_entity)

Entity marked:
 Patient diagnosed with [E]asthma[E] on 2024-08-02. Diabetes was ruled out on 2024-08-02. 


In [11]:
#Test date marking
date_text = note_text[date['start']:date['end']]
offset_date = context.find(date_text)
date_rel = {'start': offset_date, 'end': offset_date + len(date_text)}

marked_date = mark_date(context, date_rel)
print("Date marked:\n", marked_date)

Date marked:
 Patient diagnosed with asthma on [D]2024-08-02[D]. Diabetes was ruled out on 2024-08-02. 


In [12]:
#Do full pre-processing
preprocessed = preprocess_input(note_text, entity, date, window_size=20)
print("Preprocessed input:\n", preprocessed)

Preprocessed input:
 ient diagnosed with [E]asthma[[D]E] on 2024[D]-08-02. Diabetes


In [23]:
# Process samples
predictions = []

#for sample in tqdm(samples, desc="Samples"):
    #pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    #for pair in pairs:
        #text = f"Does '{pair['entity_label']}' relate to '{pair['date']}'? {sample['note_text']}..."
        #pred, conf = bert_extraction(text, model, tokenizer)
        #if pred == 1:
            #predictions.append({'entity_label': pair['entity_label'], 'date': pair['date'], 'confidence': conf})

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs:
        entity = pair['entity']
        date = pair['date_info']
        pred, conf = bert_extraction(sample['note_text'], entity, date, model, tokenizer, window_size=100)
        if pred == 1:
            predictions.append({'entity_label': entity['label'], 'date': date['parsed'], 'confidence': conf})

Samples: 100%|██████████| 101/101 [00:08<00:00, 11.43it/s]


In [24]:
#Look at predictions
predictions

[{'entity_label': 'multiple_sclerosis',
  'date': '2024-09-27',
  'confidence': 0.5034399628639221},
 {'entity_label': 'multiple_sclerosis',
  'date': '2025-01-29',
  'confidence': 0.5002328157424927},
 {'entity_label': 'congenital malformation',
  'date': '2024-09-27',
  'confidence': 0.5041528344154358},
 {'entity_label': 'depression',
  'date': '2025-05-07',
  'confidence': 0.5062710642814636},
 {'entity_label': 'copd',
  'date': '2025-05-07',
  'confidence': 0.5013790726661682},
 {'entity_label': 'diabetes_mellitus',
  'date': '2024-11-18',
  'confidence': 0.5046742558479309},
 {'entity_label': 'pituitary macroadenoma',
  'date': '2024-11-18',
  'confidence': 0.5021862983703613},
 {'entity_label': 'neoplasm of pituitary gland',
  'date': '2024-11-18',
  'confidence': 0.5002954602241516},
 {'entity_label': 'bipolar_disorder',
  'date': '2024-06-08',
  'confidence': 0.5059009790420532},
 {'entity_label': 'congenital malformation',
  'date': '2024-06-08',
  'confidence': 0.50791168212

In [25]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

{'precision': 0.24175824175824176,
 'recall': 0.205607476635514,
 'f1': 0.22222222222222224,
 'tp': 44,
 'fp': 138,
 'fn': 170}

BERT Finetuning

In [ ]:
# Prepare data
df_train = make_training_pairs(samples, gold_lookup)
dataset = Dataset.from_pandas(df_train)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
#Look at tokenized datasets
tokenized_datasets

In [ ]:
#Define training args
training_args = TrainingArguments(
    output_dir="./bert_model_training/bert_finetuned",
    per_device_train_batch_size=16,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
)

In [ ]:
#Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,  # For demo; use a real split for real training!
)

In [ ]:
#Train
trainer.train()

In [ ]:
#Process samples
predictions = []

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs:
        text = f"Does '{pair['entity_label']}' relate to '{pair['date']}'? {sample['note_text']}..."
        pred, conf = bert_extraction(text, model, tokenizer)
        if pred == 1:
            predictions.append({'entity_label': pair['entity_label'], 'date': pair['date'], 'confidence': conf})

In [ ]:
#Look at predictions
predictions

In [ ]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

LLM

In [ ]:
# Define generator
#generator = pipeline("text-generation", model="../Llama-3.2-3B-Instruct", device=-1)
generator = pipeline("text2text-generation", model="google/flan-t5-small", device=-1)

In [ ]:
# Test simple prompt using generator
prompt = "Does the following text indicate a relationship between 'asthma' and '2024-08-02'? Answer YES or NO. Text: Patient diagnosed with asthma on 2024-08-02."
result = generator(prompt)
print(result[0]['generated_text'])

In [ ]:
# Test simple prompt using llm_extraction() function
prompt = "Does the following text indicate a relationship between 'asthma' and '2024-08-02'? Answer YES or NO. Text: Patient diagnosed with asthma on 2024-08-02."
response = llm_extraction(prompt, generator)
response

In [ ]:
#Prompt to use
prompt_to_use = 'prompt.txt'

In [ ]:
#Process all date-entity pairs, make prompt, do llm extraction and make prediction
predictions = []

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    #for pair in pairs[:1]:
    for pair in pairs:
        #print(pair)
        prompt = make_binary_prompt(pair['entity'], pair['date_info'], sample['note_text'], prompt_to_use)
        #print(prompt)
        response = llm_extraction(prompt, generator)
        #print(response)
        pred, conf = parse_llm_answer(response)
        #print(pred, conf)
        if pred == 1:
            predictions.append({
                'entity_label': pair['entity_label'],
                'date': pair['date'],
                'confidence': conf
            })

In [ ]:
#Look at prediction
predictions

In [ ]:
#Calculcate metrics
metrics = calculate_metrics(predictions, df)
metrics